In [1]:
## 変数や関数の定義

from pathlib import Path
import json
from watson_developer_cloud import PersonalityInsightsV3
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN
from collections import OrderedDict
import pprint
import itertools
import get_my_tweets_over200 as tw

# 本の文章データ名
books = ["bocchan.txt", "chumonno_oi_ryoriten.txt", "hashire_merosu.txt", "kaijin_nijumenso.txt", "ningen_shikkaku.txt"]
# 抽出した本のストーリーを保存するフォルダ
stored_path = Path("./extracted_bookdata")
# 全ての本の分析結果を保存するJSONファイルの名前
analyzed_all_book_json_name = "analyzed_all_book.json"

def get_personality(text):
    """
    Personality Insightsを用いた性格分析の結果を取得する
    参考サイト：http://tmngtm.hatenablog.com/entry/2016/10/14/203901
    -> return responseで400コードが出てうまくいかず(後で分かったことだが、恐らくContent-Languageをenにしていた...)
    -> 公式マニュアルを読み解いた
    ※事前に、pip install --upgrade watson-developer-cloud
    """
    api_version = ""
    api_username = ""
    api_password = ""
    api_url = "https://gateway.watsonplatform.net/personality-insights/api"
    
    personality_insights = PersonalityInsightsV3(
        version = api_version,
        username = api_username,
        password = api_password,
        url = api_url
    )
    
    profile = personality_insights.profile(
        content = text,
        content_type = "text/plain",
        accept = "application/json",
        content_language = "ja",
        accept_language = "en",
        raw_scores = True,
    )
    
    return profile

def get_personality_old(text):
    """
    上記参考サイトのコード＋少々改変版。
    Content-Languageをjaにしていればできてたっぽい。
    """
    # endpoint url
    api_url = "https://gateway.watsonplatform.net/personality-insights/api/v2/profile"
    # username and password for this api
    api_username = ""
    api_password = ""
    
    # set query
    query = {
        "include_raw": "false",
        "header": "false"
    }
    
    # set header
    headers = {
        "Content-Type": "text/plain", # 入力テキストの形式。今回はstringのテキストを渡す設定。
        "Content-Language": "en",     # 入力言語モードの指定。jaとすると日本語の入力を受け付ける。
        "Accept": "application/json", # 応答ファイルの形式。今回はjson形式で受け取る。
        "Accept-Language": "en",      # 処理言語モードの設定。jaとすると日本語として処理する。
        "include_raw": "false"
    }
    
    # set body
    body = text
    body = body.encode("utf-8") # 参考サイトのコードに追記
    
    # post
    response = requests.post(
        api_url,
        auth = HTTPBasicAuth(api_username, api_password),
        params = query,
        headers = headers,
        data = body
    )
    return response

def convert_dict_to_json(orig_dict, indent = 4):
    """
    辞書からJSON形式に変換する
    参考サイト：https://tmg0525.hatenadiary.jp/entry/2018/03/30/204448
    """
    return json.dumps(orig_dict, indent = indent)

def load_json_as_dict(json_name):
    """
    JSON形式の文字列を辞書として読み込む
    参考サイト：https://note.nkmk.me/python-json-load-dump/
    """
    with open("./" + json_name, "r") as json_file:
        return json.load(json_file, object_pairs_hook = OrderedDict)

def sort_personality(a_book_result):
    """
    a_book_resultを性格の値で降順にソートし、リストとしてreturnする
    returnされたリストをスライスすれば一部を取得できる
    本のカテゴリ分けの時に使う
    """
    return sorted(a_book_result.items(), key = lambda x: x[1], reverse = True)

In [2]:
## 本(今回は全5冊)の文章データをPersonality Insightsに入力して分析させる
## その結果を、JSONファイルに変換して保存する
## -> このセルを1回実行しておけば、ファイルを読み込むだけで本の分析結果を利用できる

# 全ての本(今回は5冊)の分析結果を格納する
# { { book1: { personality_name1: percentile1, personality2: percentile2, ... } }, ... , { book5: { personality5: percentile5, ... } } }
# 後でJSONファイルとして保存するため、順序付き辞書を使う
# 参考サイト：https://note.nkmk.me/python-collections-ordereddict/
all_book_big5 = OrderedDict()

for book in books:
    book_path = Path(stored_path.name + "/" + book) # Path(Path("./extracted_bookdata") / book) でもOK
    # 入力となる本の文章データ
    story = book_path.read_text(encoding="utf-8")
    # personality insightsに、本の文章データを1つずつ順番に分析させる
    story_personality_dict = get_personality(story)
    
    # 本の性格
    personality_name = [big5["name"] for big5 in story_personality_dict["personality"]]
    # 性格の値。小数点第2位で四捨五入した後、decimal.Decimalからfloatに変換している。
    # 参考サイト(四捨五入)：https://note.nkmk.me/python-round-decimal-quantize/
    personality_percentile = [float(Decimal(str(big5["percentile"])).quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)) \
                              for big5 in story_personality_dict["personality"]]
    # 1冊の本の分析結果
    a_book_big5 = OrderedDict(zip(personality_name, personality_percentile))
    # 全体の分析結果の辞書に追加
    all_book_big5[book] = a_book_big5
    
    print("finished analyzing {} .".format(book))
    # JSONとして出力したくなったらコメントアウト解除
    # print(convert_dict_to_json(story_personality_dict, 4))

print("finished analyzing all({}) books.".format(len(books)))

# 全ての本の分析結果をJSONファイルとして保存する
# このノートブックをシャットダウンしたら、jsonファイルの内容を閲覧できるようになる(それまでは何故か内容が更新されていない)
fw = open(analyzed_all_book_json_name, "w")
json.dump(all_book_big5, fw, indent = 4)
print("registered result of analysis with file: {}".format(analyzed_all_book_json_name))

finished analyzing bocchan.txt .
finished analyzing chumonno_oi_ryoriten.txt .
finished analyzing hashire_merosu.txt .
finished analyzing kaijin_nijumenso.txt .
finished analyzing ningen_shikkaku.txt .
finished analyzing all(5) books.
registered result of analysis with file: analyzed_all_book.json


In [2]:
## 分析結果を基に、本のカテゴリ分けを行う。

# カテゴリの通し番号
serial_numbers = [i + 1 for i in range(10)]

# Agreeableness: 協調性, Conscientiousness: 真面目さ, Emotional range: 精神的安定性, Extraversion: 外向性, Openness: 開放性
# 参考サイト：https://note.nkmk.me/python-math-factorial-permutations-combinations/
big5_list = ["Agreeableness", "Conscientiousness", "Emotional range", "Extraversion", "Openness"]

# big5から2つの性格を選ぶ組み合わせを列挙する
# JSONファイルの内容に合わせて要素をsetに変換する(順番を考慮すると)
category_patterns = [set(pat) for pat in itertools.combinations(big5_list, 2)]

# 本ごとのカテゴリ分けを記録する辞書
all_book_category_table = OrderedDict()

for book in books:
    # 本の分析結果のJSONファイルをそれぞれ読み込み、値の大きい順にソートする
    loaded_all_book_big5 = load_json_as_dict(analyzed_all_book_json_name)[book]
    sorted_result = sort_personality(loaded_all_book_big5)
    
    # big5のうち大きい順から2番目までの値の性格名のみを取得し、その本のカテゴリとする(例：{Openness, Extraversion})
    # この時、複数の性格の順番を考慮させないために、setで用意する
    # 2つを組み合わせているため、1つだけの場合よりも幅広くカテゴリ分けできそう
    book_category = {big5_elm[0] for big5_elm in sorted_result[0:2]}
    all_book_category_table[book] = book_category

pprint.pprint(category_patterns)
pprint.pprint(all_book_category_table)
print(all_book_category_table["bocchan.txt"] in category_patterns)

[{'Agreeableness', 'Conscientiousness'},
 {'Agreeableness', 'Emotional range'},
 {'Extraversion', 'Agreeableness'},
 {'Agreeableness', 'Openness'},
 {'Conscientiousness', 'Emotional range'},
 {'Extraversion', 'Conscientiousness'},
 {'Openness', 'Conscientiousness'},
 {'Extraversion', 'Emotional range'},
 {'Openness', 'Emotional range'},
 {'Extraversion', 'Openness'}]
OrderedDict([('bocchan.txt', {'Openness', 'Emotional range'}),
             ('chumonno_oi_ryoriten.txt', {'Extraversion', 'Openness'}),
             ('hashire_merosu.txt', {'Extraversion', 'Openness'}),
             ('kaijin_nijumenso.txt', {'Agreeableness', 'Openness'}),
             ('ningen_shikkaku.txt', {'Agreeableness', 'Openness'})])
True


In [2]:
## 自分のツイートをAPIに入力して結果を見る
## 一番上のセルでget_my_tweets_over200をimport済み

# ツイートの取得
tw.userTweets = [] # import元にある、取得したツイートの格納リストの中身をリセットする(これで繰り返し実行してもOK)
tw.use_function()

print(tw.userTweets)
print()
print(len(tw.userTweets))
print()

userTweets_joined = " ".join(tw.userTweets)
print(userTweets_joined)

# 自分のツイートをPersonality Insightsに入れて結果を見る
result = get_personality(userTweets_joined)
print(convert_dict_to_json(result, 4))

自分のuser_idを入力してください
>> sftn_sk
----------------------------------------------------
aa
1022000600752513025
bb
ツイートの取得が終了しました

['今日めっちゃ近くの場所にあゆタロウさんいたんか…(°_°)', 'LINE通話中にいきなりPCの電源切れてびびった', '実際の譜面見てみたけど、これで星9でええやろ\n\nhttps://t.co/04mP99Rm0I', 'RT @amanao_: @XDAY1206 実行してみました！ https://t.co/Px9C8xl8JF', 'RT @XDAY1206: void setup 公式歌詞\n公式....歌....詞....？ https://t.co/AJrQW1b1n3', 'キムドンフン引退ってマジ！？\n衝撃…', '名前いかつい笑\nグ  ラ  ン  デ  ュ  ー  ク https://t.co/gXFLTxOXog', '布団でノートPC使ってたら寝落ちしてた', 'おージョコビッチ優勝か', 'エルモがエルモにリプ送ってて草 https://t.co/PKThdMp9KJ', '黒川、徹底して幹事拒否してるの笑う', 'あっっっつ', '全米オープン、大坂なおみが優勝で錦織ベスト4とか凄すぎ', 'すげえ、TrelloってiOSアプリ版もあるのか', 'GitLFS導入できぬ', '錦織劣勢の裏で大谷19号HR打ってるw', 'バイト面接終わりー', 'グアム旅行で朝早かったおかげで、目覚め良くなった気がする', 'グアムロス', '逆転裁判のアニメ続編やるのか！', '家着いたんで片付けします😭', '初海外、楽しかったし新鮮だった🌴', '@riiiiku08 マンゴーはクセが強いかも。\n\nそれは辛い…\nバラ売りしてないところもあるのか(-_-;)', '@riiiiku08 個人的には飲みやすかったけど、林檎の方が断然良かった！\n\n6缶もあると消費が大変そう…笑', '改めて路線図見たら、ほぼ始発駅〜終点まででした', '京成線の乗車時間なげえ', 'いよいよ帰国するナリ', '🍺 https://t.co/Bkjc0fV0LJ', 'バス乗った時、$10余分に渡してし

{
    "word_count": 8201,
    "processed_language": "ja",
    "personality": [
        {
            "trait_id": "big5_openness",
            "name": "Openness",
            "category": "personality",
            "percentile": 0.26615270771895105,
            "raw_score": 0.5713597418775663,
            "significant": true,
            "children": [
                {
                    "trait_id": "facet_adventurousness",
                    "name": "Adventurousness",
                    "category": "personality",
                    "percentile": 0.7060027236315843,
                    "raw_score": 0.5941583788255782,
                    "significant": true
                },
                {
                    "trait_id": "facet_artistic_interests",
                    "name": "Artistic interests",
                    "category": "personality",
                    "percentile": 0.18269227004493754,
                    "raw_score": 0.6319304569037626,
                    "significa